In [1]:
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting, BaseBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore
from dfg_rating.model.evaluators.accuracy import Likelihood
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.forecast.base_forecast import SimpleForecast, BaseForecast
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.network.base_network import BaseNetwork
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction

#specify rating error
from dfg_rating.model.rating.base_rating import RatingFunctionError
rating_error = RatingFunctionError(error='normal', loc=0, scale=50)

import networkx as nx
import pandas as pd
import numpy as np

from dfg_rating.model.network.simple_network import RoundRobinNetwork

pd.set_option('display.max_columns', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
network: BaseNetwork = factory.new_network(
    'multiple-round-robin',
    teams=60,
    days_between_rounds=3,
    seasons=10,
    league_teams=60,
    league_promotion=0,
    create=True,
    true_forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients = [-0.9,0.3], beta_parameter=0.006),
    true_rating=ControlledTrendRating(starting_point=ControlledRandomFunction(distribution='normal', loc=1000, scale=100),
                                     delta=ControlledRandomFunction(distribution='normal', loc=0, scale=3),
                                     trend=ControlledRandomFunction(distribution='normal', loc=0, scale=20/365),
                                     season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=10))
)

#network.create_data()

Season 0
Relegation candidates {54: 41.0, 18: 42.0, 4: 43.0, 27: 46.0, 59: 47.0, 19: 47.0}
Relegation teams []
Season 1
Relegation candidates {20: 32.0, 46: 38.0, 54: 40.0, 16: 40.0, 19: 43.0, 4: 44.0}
Relegation teams []
Season 2
Relegation candidates {4: 27.0, 46: 28.0, 54: 33.0, 52: 38.0, 26: 42.0, 59: 46.0}
Relegation teams []
Season 3
Relegation candidates {4: 20.0, 59: 31.0, 46: 32.0, 16: 32.0, 26: 36.0, 19: 36.0}
Relegation teams []
Season 4
Relegation candidates {46: 21.0, 26: 27.0, 4: 30.0, 19: 35.0, 52: 38.0, 16: 38.0}
Relegation teams []
Season 5
Relegation candidates {46: 13.0, 26: 16.0, 3: 36.0, 16: 36.0, 19: 38.0, 59: 40.0}
Relegation teams []
Season 6
Relegation candidates {26: 13.0, 46: 17.0, 19: 27.0, 4: 27.0, 16: 33.0, 32: 35.0}
Relegation teams []
Season 7
Relegation candidates {46: 16.0, 26: 26.0, 59: 35.0, 50: 36.0, 16: 41.0, 19: 42.0}
Relegation teams []
Season 8
Relegation candidates {26: 14.0, 46: 21.0, 4: 23.0, 40: 31.0, 32: 33.0, 16: 33.0}
Relegation teams []


In [3]:
#network.print_data(schedule=True)

In [5]:
#adding biased bookmaker forecast
network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-1.2, 0.0], beta_parameter=0.006, home_team_error=rating_error, away_team_error=rating_error),
    forecast_name='bookmaker_forecast',
    base_ranking='true_rating'
)

In [6]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=0.10)
)
network.add_odds(
    bookmaker_name="bm",
    bookmaker=bookmaker,
    base_forecast='bookmaker_forecast'
)

In [7]:
betting = FixedBetting(100)

In [8]:
def aggregate_measures(list_of_matches, prefix=''):
    return {prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]), 
           prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]), 
           prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
           prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches])}

In [9]:
result_list = []
for c0 in np.arange(-1.0, 0.6, 0.1):
    for c1 in np.arange(c0+0.2, 0.8, 0.1):
        for beta in np.arange(0.004, 0.010, 0.002):
            
            print(f'Variables {c0}, {c1} and {beta}')
            
            rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
            betting_returns = BettingReturnsEvaluator(outcomes=['home', 'draw', 'away'], player_name = 'b', true_model = 'true_forecast', bookmaker_name = 'bm')
            likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
            
            network.add_forecast(
            forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[c0, c1], beta_parameter=beta, home_team_error=rating_error, away_team_error=rating_error),
            forecast_name='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta),
            base_ranking='true_rating'
            )
                
            network.add_bets(
            bettor_name='b',
            bookmaker='bm',
            betting=betting,
            base_forecast='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)
            )
            
            network.add_evaluation(rps, 'rps')
            network.add_evaluation(betting_returns, 'betting_returns')
            network.add_evaluation(likelihood, 'likelihood')
        
            all_matches = [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games()]
            is_matches = [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] < 4]
            oos_matches = [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > 3]
            
            result_all = aggregate_measures(all_matches, 'all')
            result_is = aggregate_measures(is_matches, 'is')
            result_oos = aggregate_measures(oos_matches, 'oos')
            result = {
                'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos, **result_all
            }
            
            result_list.append(result)
            
        #network.serialize_network('economic_network')
        
#network.export(forecasts = ['bookmaker_forecast', 'true_forecast', 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)],
#                      metrics = ['rps', 'betting_returns', 'loglikelihood'],
#                      odds=['bm'])
            
            

            

Variables -1.0, -0.8 and 0.004
Variables -1.0, -0.8 and 0.006
Variables -1.0, -0.8 and 0.008
Variables -1.0, -0.7000000000000001 and 0.004
Variables -1.0, -0.7000000000000001 and 0.006
Variables -1.0, -0.7000000000000001 and 0.008
Variables -1.0, -0.6000000000000001 and 0.004
Variables -1.0, -0.6000000000000001 and 0.006
Variables -1.0, -0.6000000000000001 and 0.008
Variables -1.0, -0.5000000000000001 and 0.004
Variables -1.0, -0.5000000000000001 and 0.006
Variables -1.0, -0.5000000000000001 and 0.008
Variables -1.0, -0.40000000000000013 and 0.004
Variables -1.0, -0.40000000000000013 and 0.006
Variables -1.0, -0.40000000000000013 and 0.008
Variables -1.0, -0.30000000000000016 and 0.004
Variables -1.0, -0.30000000000000016 and 0.006
Variables -1.0, -0.30000000000000016 and 0.008
Variables -1.0, -0.20000000000000018 and 0.004
Variables -1.0, -0.20000000000000018 and 0.006
Variables -1.0, -0.20000000000000018 and 0.008
Variables -1.0, -0.1000000000000002 and 0.004
Variables -1.0, -0.10000

In [10]:
df = pd.DataFrame(result_list)
df.to_excel("Results_Economics.xlsx")

In [10]:
for a,h,m_id,attributes in network.iterate_over_games():
    print([v for v in attributes['metrics']['betting_returns']])

[(0.0, -0.0), (-0.0, -0.0), (-1.0, -0.050000000000000044)]
[(-1.0, -0.050000000000000044), (0.0, -0.0), (-1.0, -0.050000000000000044)]
[(-1.0, -0.050000000000000044), (-0.0, -0.0), (1.4262599306946533, -0.050000000000000044)]
[(-0.0, -0.0), (-0.0, -0.0), (2.5176962024058818, -0.04999999999999993)]
[(-0.0, -0.0), (-0.0, -0.0), (1.8450524232756682, -0.050000000000000044)]
[(-0.0, -0.0), (-0.0, -0.0), (0.9093817052671449, -0.050000000000000155)]
[(-1.0, -0.04999999999999993), (0.0, -0.0), (-1.0, -0.04999999999999993)]
[(0.5172996638136671, -0.050000000000000044), (-0.0, -0.0), (-1.0, -0.04999999999999993)]
[(0.0, -0.0), (-0.0, -0.0), (-1.0, -0.050000000000000155)]
[(-0.0, -0.0), (0.0, -0.0), (-1.0, -0.050000000000000044)]
[(-1.0, -0.050000000000000044), (0.0, -0.0), (-0.0, -0.0)]
[(-0.0, -0.0), (-0.0, -0.0), (1.1944135568743275, -0.04999999999999993)]
[(0.0, -0.0), (-0.0, -0.0), (-1.0, -0.050000000000000155)]
[(0.0, -0.0), (-0.0, -0.0), (-1.0, -0.050000000000000044)]
[(0.0, -0.0), (-0.0, 